In [2]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import requests
import os
import glob
import pandas as pd

In [7]:
!pip3 install torch torchvision

  Using cached torch-2.0.1-cp39-none-macosx_10_9_x86_64.whl (143.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.6 MB/s eta 0:00:0000:0100:01


In [6]:
# load image from the IAM database (actually this model is meant to be used on printed text)
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-printed')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-printed')

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.weight', 'encoder.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
labels_csv = pd.read_csv('/Users/rsoedarnadi/Documents/GitHub/bdc-team4/DataTestLabelled.csv', delimiter=',')
list_license_plates = labels_csv['Image Labels']
list_file_names = labels_csv['Name of File']

In [31]:
generated_plate_numbers = []

In [36]:
for file in list_file_names[55:]:
    path = os.path.join(os.getcwd(), 'Test-Data', file)
    image = Image.open(path)
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    generated_plate_numbers.append(generated_text)

In [34]:
path = os.path.join(os.getcwd(), 'Test-Data', 'DataTest55.jpg')
image = Image.open(path)
pixel_values = processor(images=image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_plate_numbers.append(generated_text)

In [37]:
import re

In [35]:
len(generated_plate_numbers)

55

In [38]:
generated_plate_numbers_ = generated_plate_numbers

In [39]:
for i, platenum in enumerate(generated_plate_numbers_):
   generated_plate_numbers_[i] = re.sub('[^A-Za-z0-9]+', '', platenum)

In [42]:
import numpy as np

In [66]:
def calculate_predicted_accuracy(names, actual_list, predicted_list):
    list_accuracy = []
    for name, actual_plate, predict_plate in zip(names, actual_list, predicted_list):
        accuracy = 0
        num_matches = 0
        if actual_plate == predict_plate:
            accuracy = 1.0
        else:
            for a, p in zip(actual_plate, predict_plate):
                if a == p:
                    num_matches += 1
            accuracy = round((num_matches / len(actual_plate)), 2)
        list_accuracy.append(accuracy)
    list_accuracy = np.asarray(list_accuracy)
    avg_value = np.average(list_accuracy)
    return avg_value

In [67]:
calculate_predicted_accuracy(list_file_names, list_license_plates, generated_plate_numbers_)

0.6053000000000001